<a href="https://colab.research.google.com/github/ImagingDataCommons/IDC-Tutorials/blob/master/notebooks/advanced_topics/clinical_data_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IDC clinical data exploration

The goal of this notebook is to introduce the users of NCI Imaging Data Commons (IDC) to the organization of clinical data BigQuery tables that accompany some of the IDC imaging data. For a quick visual summary of the clinical data available in IDC, please check out [this DataStudio dashboard](https://datastudio.google.com/u/0/reporting/04cf5976-4ea0-4fee-a749-8bfd162f2e87/page/p_s7mk6eybqc).

[NCI Imaging Data Commons (IDC)](https://imaging.datacommons.cancer.gov) is a cloud-based repository of publicly available cancer imaging data co-located with the analysis and exploration tools and resources. IDC is a node within the broader [NCI Cancer Research Data Commons (CRDC)](https://datacommons.cancer.gov/) infrastructure that provides secure access to a large, comprehensive, and expanding collection of cancer research data.

If you are not familiar with IDC, we recommend you first take a look at the [Getting started](https://github.com/ImagingDataCommons/IDC-Tutorials/blob/master/notebooks/getting_started/) notebooks that are intended to serve as the introduction into working with IDC programmatically.

If you have any questions about this tutorial, please post your questions on the [IDC user forum](https://discourse.canceridc.dev/) (preferred) or email IDC support at support@canceridc.dev!

Authored by Andrey Fedorov and George White

Prepared: July 2022

Updated: May 2023

# Prerequisites

In order to be able to run the cells in this notebook, you must complete the prerequisites to set up your Google Cloud Platform account, as shown in this tutorial: https://github.com/ImagingDataCommons/IDC-Examples/blob/master/notebooks/getting_started/part1_prerequisites.ipynb.

Once you completed the prerequisites, insert your Google Cloud Platform project ID in the cell below in place of `REPLACE_ME_WITH_YOUR_PROJECT_ID`.

In [ ]:
#@title Enter your Project ID
# initialize this variable with your Google Cloud Project ID!
my_ProjectID = "" #@param {type:"string"}

import os
os.environ["GCP_PROJECT_ID"] = my_ProjectID

!gcloud config set project $GCP_PROJECT_ID

In the following cell you will be asked to authorize Google Collaboratory to act on your behalf. In order to proceed with the following cells you must allow this.

In [2]:
# you will need to authenticate with your Google ID to do anything meaningful with IDC
from google.colab import auth
auth.authenticate_user()

# Clinical data - background

By clinical data we refer to the broad spectrum of image-related data that may accompany images. Such data may include demographics of the patients, observations related to their clinical history (therapies, diagnoses, findings), lab tests, surgeries.

Clinical data is often critical in understanding imaging data, and is essential for the development and validation of imaging biomarkers. However, such data is most often stored in spreadsheets that follow conventions specific to the site that collected the data, may not be accompanied by the dictionary defining the terms used in describing clinical data, and is rarely harmonized. As an example, you can consider examing the clinical data that accompanies the [ACRIN 6698 collection](https://wiki.cancerimagingarchive.net/pages/viewpage.action?pageId=50135447) curated by The Cancer Imaging Archive (TCIA). File named "Full Ancillary Patient Information file.xlsx" linked from the collection page contains two sheets, one of which is the dictionary, and the othe one contains per-patient data with the columns defined by the dictionary.

Not only the terms used in the clinical data accompanying individual collection are not harmonized, but the format of the spreadsheets is also collection-specific. In order to search and navigate clinical data, one has to parse those collection specific tables, and there is no interface to support searching across collections.

With the release v11 of IDC, we make the attempt to lower the barriers for accessing clinical data accompanying IDC imaging collections. We parse collection-specific tables, and organize the underlying data into BigQuery tables that can be accessed using standard SQL queries. You can also see the summary of clinical data available for IDC collections in [this dashboard](https://datastudio.google.com/u/0/reporting/04cf5976-4ea0-4fee-a749-8bfd162f2e87/page/p_s7mk6eybqc). 

At the completion of this tutorial you will learn how IDC clinical data is organized, and how to write queries to interrogate this data.

# Organization of clinical data in IDC

Clinical data accompanying imaging collections is organized into collection-specific tables in the `idc_current_clinical` BigQuery dataset (in order to locate this dataset, you will need to navigate to the [BigQuery console](https://console.cloud.google.com/bigquery), and locate `idc_current_clinical` under the `bigquery-public-data` project, which you should have added in the prerequisites mentioned earlier. 

When a certain imaging collection is accompanied by a single clinical sheet, the BigQuery table will be named `<collection_id>_clinical`. For the collection that have multiple clinical sheets (saved as either separate files, or separate Excel sheets), the tables will be named `<collection_id>_<sheet_id>`. 

## Inventory of clinical data tables: `table_metadata`

In [ ]:
# python API is the most flexible way to query IDC BigQuery metadata tables
from google.cloud import bigquery
bq_client = bigquery.Client(my_ProjectID)

selection_query = """
SELECT *
FROM bigquery-public-data.idc_current_clinical.table_metadata
ORDER BY collection_id
"""

selection_result = bq_client.query(selection_query)
table_metadata_df = selection_result.result().to_dataframe()

table_metadata_df

The per-row attributes stored in this table aim to describe the content of the specific clinical table, its versioning and provenance (what specific file/sheet was used to populate the content of the table).

Column `table_name` contains the fully resolved pointer to the BigQuery table with the parsed clinical data.

## Inventory of clinical data "dictionary terms": `column_metadata`

The purpose of the clinical data "dictionary terms" is to to facilitate exploration of and access to the clinical data. Although the content of the clinical data sheets is for the most part not harmonized, they follow the same general structure:

1. files/sheets that contain per-patient rows of clinical data attributes, defined by column headers
2. (in some cases) file/sheet defining the meaning of the individual columns and (in some cases, and when applicable) the dictionary defining the enumerated values for the specific

The IDC inventory of the clinical data "dictionary terms" contains the complete list of column names, across all clinical sheets, and per-column attributes to facilitate exploration. Let's look at the content to better understand what this means.

In [ ]:
selection_query = """
SELECT *
FROM bigquery-public-data.idc_current_clinical.column_metadata
ORDER BY collection_id
"""

selection_result = bq_client.query(selection_query)
column_metadata_df = selection_result.result().to_dataframe()

In [ ]:
column_metadata_df.columns

The only column that is initialized consistently across those collection-specific tables is `dicom_patient_id`, which corresponds to the `PatientID` column in the `dicom_all` table. `dicom_patient_id` is the only column that is curated by IDC, since the original clinical data tables do not follow the consistent convention while referring to the patient identification.

Note that some imaging collections do not have any clinical data at all, while others may have multiple tables containing various types of clinical data.

`column_metadata` should be used as a "key" for identifying information of interest in clinical metadata. It contains information about the columns across all of the collections for which we have clinical data.

The most important columns in this table are the following:
* `column`: name of the column in a collection-specific clinical metadata table; note that for some of the collections this name may not contain any human readable information, and may instead be a coded value
* `column_label`: for some collections it may contain a description for a column name that otherwise may not be human readable. For other collections it is the original name of the column in the source Excel file
* `collection_id` and `table_name` identify the collection the specific column corresponds to, and the name of the clinical data table where this column is present.



Here's an example of the columns that correspond to the clinical metadata in one of the ACRIN collections, where `column` is not particularly helpful, but `column_label` provides human readable information to allow interpretation of the column.

In [ ]:
column_metadata_df[column_metadata_df["collection_id"] == "acrin_nsclc_fdg_pet"][:5]

For some columns, the values come from a defined set. In the example above, we can, for example, examine the values assigned to encode patient race.

In [ ]:
race_values = column_metadata_df[(column_metadata_df["collection_id"] == "acrin_nsclc_fdg_pet") & (column_metadata_df["column_label"] == "RACE")]["values"]

race_values.tolist()

On the other hand, if we look at the metadata available for the `c4kc_kits` collection, `column_label` and `column` are identical. 


In [ ]:
column_metadata_df[column_metadata_df["collection_id"] == "c4kc_kits"][:3]

**As a general rule of thumb**:
* when selecting specific columns from clinical tables, use `column` values
* when searching for concepts of interest in `column_metadata`, use `column_label`

# Exploring IDC clinical data

In the following sections of the notebook we go over some use cases to demonstrate various options for navigating IDC clinical data. 

As always, if you have a use case that is not addressed here, if you have suggestions or are confused - please start a discussion thread on the [IDC User forum](https://discourse.canceridc.dev/)!

## Collection-focused exploration

If you used clinical data stored in TCIA, perhaps you started with a specific collection of your interest, downloaded clinical data files for that collection, parsed them into your code. Let's go over those steps the IDC way!

First, let's see which of the collections in IDC have clinical data in BigQuery. To do that we will reuse the pandas dataframe with the results of the query we executed earlier.


In [ ]:
column_metadata_df["collection_id"].unique().tolist()

If you are interested in what clinical data is available for the specific collection, you can select only the rows corresponding to that collection in the `column_metadata` table. Here we select a subset of columns to improve readability of the dataframe.

Note that for some collections, clinical data sheets are accompanied by dictionaries, which formalize the values encountered. Examples of such collections are [ISPY1](https://wiki.cancerimagingarchive.net/display/Public/ISPY1) or ACRIN trials.

For many other collections there are no such dictionaries available. In those situations, the values you will see in the `values` columns have been derived by examining the distinct values encountered in the clinical data sheets.

In the following we look at the clinical data columns ("dictionary terms") for the [ACRIN 6698 collection](https://wiki.cancerimagingarchive.net/pages/viewpage.action?pageId=50135447) mentioned earlier.

In [ ]:
acrin6698_clinical_df = column_metadata_df[column_metadata_df["collection_id"] == "acrin_6698"] 

acrin6698_clinical_df

Here's how you can select just the specific columns in the table - this way it is easier to examine the data.

In [ ]:
acrin6698_clinical_df[["collection_id", "table_name", "column", "column_label", "data_type", "values"]]

`table_name` gives us the location of the BigQuery table that contains the column described in the `column_metadata` row.

Given the information available in the per-collection clinical data, we can proceed with selecting a subset of patients that meet the criteria of your interest. As an example, the following query will select all of the distinct combination of patient ID and tumor grade, as defined by the `sbrgrade` column contents.

Note that in this case we use `%%bigquery` magic to execute the query - this is done just to demonstrate an alternative ways to work with SQL. You can switch between this approach and BigQuery Python API based on your preference! 

When you use `%%bigquery` magic, the result of the query will be saved into a pandas dataframe you specify as the last argument to the call (`acrin_6698_tumors`, in the example below). If you skip that argument, the result will be displayed in the cell, but you cannot reuse it later.

In [ ]:
%%bigquery acrin_6698_tumors --project=$my_ProjectID

SELECT 
  DISTINCT(dicom_patient_id), sbrgrade
FROM 
  bigquery-public-data.idc_current_clinical.acrin_6698_clinical


In [ ]:
acrin_6698_tumors

We can next use `dicom_patient_id` to link clinical data with the imaging studies available for the given patient. The query becomes a bit more complex, since we need to join data across two tables.

In [ ]:
%%bigquery acrin_6698_viewable --project=$my_ProjectID 

SELECT
  ANY_VALUE(PatientID) AS PatientID,
  STRING_AGG(DISTINCT(acrin_6698_clinical.sbrgrade)) as tumor_grade,
  STRING_AGG(DISTINCT(Modality)) AS modalities,
  COUNT(DISTINCT(SeriesInstanceUID)) AS num_series,
  ANY_VALUE(CONCAT("https://viewer.imaging.datacommons.cancer.gov/viewer/", StudyInstanceUID)) AS viewer_url
FROM
  bigquery-public-data.idc_current.dicom_all AS dicom_all
JOIN
  bigquery-public-data.idc_current_clinical.acrin_6698_clinical AS acrin_6698_clinical
ON
  dicom_all.PatientID = acrin_6698_clinical.dicom_patient_id
GROUP BY
  StudyInstanceUID
ORDER BY
  PatientID

In [ ]:
acrin_6698_viewable

Remember that Google Cloud [BigQuery console](https://console.cloud.google.com/bigquery) is very handy in interactive exploration of tables and debugging of the queries! Until you become familiar with BigQuery and SQL, it is not recommended that you debug queries from the notebook cells. 

As an exercise, try to modify the query above to include count of distinct values of `SOPInstanceUID` in the query result (that number will correspond to the number of files you would be downloading for a given study).

## Discovery mode

Sometime you may want to find whether specific clinical attribute is available for the imaging data you can find in IDC. 

We can start by looking at the distinct values of `column_label` (which in the general case will be either more descriptive, or identical to `column`).


In [ ]:
import pandas as pd

pd.DataFrame({"column_label":column_metadata_df["column_label"].unique()})

Let's say we want to know which of the cases have information related to therapy. We can search column metadata for the presence of word "therapy" (since the terms in clinical data are not harmonized, we need to account for the variability in capitalization).

In [ ]:
column_metadata_df[column_metadata_df["column_label"].str.contains("[tT]herapy", na=False)][["column_label", "collection_id", "values"]]

We observe that there are several collections that contain column named "Chemotherapy". Let's filter these values further, in order to identify subjects that underwent chemotherapy.

In [ ]:
column_metadata_df[column_metadata_df["column_label"].str.contains("[Cc]hemotherapy", na=False)][[ "collection_id", "table_name", "column", "column_label","values"]]

From the table above we can observe that:

1. There are several collections that have clinical metadata related to chemotherapy regimen of the subject; 

2. There are no value sets assigned for the column `induction_chemotherapy` in the table `hnscc_clinical`. Value sets may be missing if the dictionary is not provided, or if there are a large number (>20) distinct values of the column in the table. In such cases one can check the distinct values of the column by querying the table directly.  

Looking at the value sets for the collections/columns that have those, we can observe that subjects that had any chemotherapy could be selected as follows for the respective collections (non-exhaustive list):
* `head_neck_radiomics_hn1`: subjects that have value other than `none` in table `head_neck_radiomics_hn1_clinical` column `chemotherapy_given`
* `hnscc`: subjects that have value other than `No` in table `hnscc_clinical` column `chemotherapy_regime`
* `hnscc_3dct_rt`: subjects that have value other than `None` in table `hnscc_3dct_rt_demographics` column `chemotherapy_medication`
* `nsclc_radiogenomics`: subjects that have value `Yes` in table `nsclc_radiogenomics_clinical` column `chemotherapy`
* `hcc_tace_seg` column `chemotherapy` in the `hcc_tace_seg_clinical` table

Let's focus on the clinical data related to chemotherapy for the collection `hcc_tace_seg`.

In [ ]:
chemotherapy_subset = column_metadata_df[column_metadata_df["column_label"].str.contains("[Cc]hemotherapy", na=False)][[ "collection_id", "table_name", "column", "column_label","values"]]

chemotherapy_subset = chemotherapy_subset[chemotherapy_subset["collection_id"] == "hcc_tace_seg"]

chemotherapy_subset

Before we select subjects that meet the criteria defined above, let's confirm  the values encountered in table `hcc_tace_seg_clinical` column `chemotherapy` match the value set in the `column_metadata` table. Here are the values of the dictionary we observe from the `column_metadata` inventory.

In [ ]:
chemotherapy_subset["values"].tolist()

The query below will select the distinct values encountered in the `chemotherapy` column of the `hcc_tace_seg_clinical` table, which we confirm matches those in the dictionary.

In [ ]:
selection_query = """
SELECT DISTINCT(chemotherapy)
FROM bigquery-public-data.idc_current_clinical.hcc_tace_seg_clinical
"""

selection_result = bq_client.query(selection_query)
hcc_tace_seg_clinical_df = selection_result.result().to_dataframe()
hcc_tace_seg_clinical_df

Next we can examine the data to see what therapy individual patients had.

In [ ]:
selection_query = """
SELECT DISTINCT(dicom_patient_id), chemotherapy
FROM bigquery-public-data.idc_current_clinical.hcc_tace_seg_clinical
WHERE chemotherapy <> "null"
"""

selection_result = bq_client.query(selection_query)
hcc_tace_seg_clinical_df = selection_result.result().to_dataframe()
hcc_tace_seg_clinical_df

`dicom_patient_id` is the key to connect clinical data and imaging data. Let's see what imaging studies we have for patient `HCC_101` that was treated with 	doxorubicin LC beads. Along the way we can also generate viewer URLs to conveniently examine the images.

In [ ]:
%%bigquery --project=$my_ProjectID

SELECT 
  StudyInstanceUID, 
  STRING_AGG(DISTINCT(Modality)) AS modalities, 
  STRING_AGG(DISTINCT(collection_id)) AS collection_id, 
  STRING_AGG(DISTINCT(Access)) AS access,
  COUNT(DISTINCT(SeriesInstanceUID)) AS num_series,
  ANY_VALUE(CONCAT("https://viewer.imaging.datacommons.cancer.gov/viewer/", StudyInstanceUID)) as viewer_url
FROM bigquery-public-data.idc_current.dicom_all
WHERE PatientID = "HCC_103"
GROUP BY StudyInstanceUID

## Patient-focused exploration

Sometime you may want to know whether specific patient has any clinical data available. One way to do this is to locate the collection that patient belongs to, and then check whether any of the clinical data tables (if any) that are available for that collection have that patient identifier.

Alternatively, we can build a complete list of patients that have clinical data by performing a union on all of the `dicom_patient_id` columns across all of the clinical data tables, which is what we do in the next cell.

In this query we use the ability of BigQuery to [query multiple tables using a wildcard table](https://cloud.google.com/bigquery/docs/querying-wildcard-tables). Note that here we refer to the specific version of the data, since `idc_current_clinical` dataset contains views, which cannot be queried through prefix.

In [ ]:
import re

selection_query = """
SELECT
  DISTINCT(dicom_patient_id)
FROM
  `bigquery-public-data.idc_v14_clinical.*`
WHERE
  _TABLE_SUFFIX NOT IN ("table_metadata",
    "column_metadata" )
"""

selection_result = bq_client.query(selection_query)
patients_df = selection_result.result().to_dataframe()

patients_df


# Want to learn more?

* check out other notebooks: https://github.com/ImagingDataCommons/IDC-Examples/tree/master/notebooks
* join our community forum to ask any questions about IDC: https://discourse.canceridc.dev/
* ask your questions during live discussions with IDC developers at the IDC weekly office hours - join us on Google Meet at https://meet.google.com/xyt-vody-tvb every Tuesday 16:30 – 17:30 (New York) and Wednesday 10:30-11:30 (New York)
* browse IDC portal: https://imaging.datacommons.cancer.gov/explore/
* read IDC paper: http://dx.doi.org/10.1158/0008-5472.CAN-21-0950
* watch a recent presentation about IDC: https://youtu.be/P9ateg9ZUEs